In [2]:
import pandas as pd
import os
import re
import spacy
from gensim.models.phrases import Phrases, Phraser
from time import time 
import multiprocessing
from gensim.models import Word2Vec
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [3]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [4]:
df=pd.read_csv('train.csv')

In [21]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df.shape

In [3]:
X=df[['text']]
y=df[['target']]

In [4]:
description_list = df['text'].tolist()
text=np.array(df['target'].tolist())

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pickle
count_vect = CountVectorizer()
#count_vect._validate_vocabulary()
x_train_counts = count_vect.fit_transform(description_list)


tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

# Save the vectorizer
vec_file = 'vectorizer.pickle'
pickle.dump(count_vect, open(vec_file, 'wb'))

    # Save the model
# mod_file = 'classification.model'
# pickle.dump(model, open(mod_file, 'wb'))

In [ ]:
x_train_tfidf.shape

In [21]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train_tfidf,text)

In [30]:
df_test=pd.read_csv('test.csv')

In [23]:
description_list2 = df_test['text'].tolist()

In [25]:
loaded_vectorizer = pickle.load(open('vectorizer.pickle', 'rb'))
out=clf.predict(loaded_vectorizer.transform(description_list2))

In [27]:
output=pd.DataFrame(out,columns=['target'])

In [29]:
test=pd.concat([df_test,output],axis=1)

In [30]:
test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [31]:
test.to_csv('output.csv',index=False)

In [ ]:
out=clf.predict(x_train_tfidf2)

In [ ]:
x_train_tfidf2.shape

In [ ]:
df_test.shape

In [ ]:
df.shape

In [36]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/2" 
embed = hub.load(module_url)

In [48]:
np_list = np.asarray(description_list)
tensor_list = tf.convert_to_tensor(np_list)

In [ ]:
import tensorflow as tf
with tf.compat.v1.Session() as session:
    session.run([tf.compat.v1.global_variables_initializer(), 
                 tf.compat.v1.tables_initializer()])
training_embeddings = session.run(embed.signatures['default'](tensor_list))

In [ ]:
model = Sequential()
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

In [16]:
import gensim.models as mad
wv = mad.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
wv.init_sims(replace=True)

In [5]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [8]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    

train_tokenized = df.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)


TypeError: ('w2v_tokenize_text() takes 1 positional argument but 2 were given', 'occurred at index 0')

In [28]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, df['target'])



C:\Users\dmsss\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
y_pred = logreg.predict(X_test_word_average)

In [ ]:
y_pred.to_csv('jujubi.csv')

In [31]:
test_tokenized = df_test.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values
X_test_word_average = word_averaging_list(wv,test_tokenized)

C:\Users\dmsss\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [32]:
y_pred = logreg.predict(X_test_word_average)

In [33]:
y_pred

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [34]:
output=pd.DataFrame(y_pred,columns=['target'])

In [35]:
test=pd.concat([df_test,output],axis=1)

In [36]:
test.to_csv('output.csv',index=False)

In [1]:
test

NameError: name 'test' is not defined